## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1=tf.compat.v1.get_variable(shape=(784,1000),name="W1",initializer=tf.compat.v1.random_normal_initializer)
        self.b1=tf.compat.v1.get_variable(shape=(1,1000),name="b1",initializer=tf.compat.v1.random_normal_initializer)
        self.W2=tf.compat.v1.get_variable(shape=(1000,666),name="W2",initializer=tf.compat.v1.random_normal_initializer)
        self.b2=tf.compat.v1.get_variable(shape=(1,666),name="b2",initializer=tf.compat.v1.random_normal_initializer)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x=tf.reshape(x,(-1,784))
        hidden = tf.nn.relu(tf.matmul(x,self.W1)+self.b1) #RELU函数
        logits = tf.matmul(hidden,self.W2)+self.b2 # 双层RELU
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 583.79175 ; accuracy 0.00045
epoch 1 : loss 523.9974 ; accuracy 0.0019166666
epoch 2 : loss 471.9332 ; accuracy 0.0043833335
epoch 3 : loss 423.13644 ; accuracy 0.0085166665
epoch 4 : loss 377.12933 ; accuracy 0.014983334
epoch 5 : loss 334.5906 ; accuracy 0.0257
epoch 6 : loss 296.71286 ; accuracy 0.04238333
epoch 7 : loss 264.18048 ; accuracy 0.0636
epoch 8 : loss 236.63004 ; accuracy 0.0848
epoch 9 : loss 213.03996 ; accuracy 0.106183335
epoch 10 : loss 192.49184 ; accuracy 0.12918334
epoch 11 : loss 174.53227 ; accuracy 0.15188333
epoch 12 : loss 158.99147 ; accuracy 0.17391667
epoch 13 : loss 145.92252 ; accuracy 0.1969
epoch 14 : loss 135.21548 ; accuracy 0.21775
epoch 15 : loss 126.35385 ; accuracy 0.23855
epoch 16 : loss 118.894264 ; accuracy 0.25786668
epoch 17 : loss 112.49187 ; accuracy 0.27743334
epoch 18 : loss 106.926384 ; accuracy 0.29601666
epoch 19 : loss 102.0514 ; accuracy 0.31483334
epoch 20 : loss 97.741234 ; accuracy 0.33283332
epoch 21 : loss 93.89

In [6]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    if epoch%20==0:
        print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 50.251812 ; accuracy 0.5818833
epoch 20 : loss 40.728855 ; accuracy 0.64348334
epoch 40 : loss 35.144302 ; accuracy 0.68221664
epoch 60 : loss 31.374804 ; accuracy 0.70916665
epoch 80 : loss 28.637985 ; accuracy 0.7298
epoch 100 : loss 26.526978 ; accuracy 0.7458
epoch 120 : loss 24.832962 ; accuracy 0.75913334
epoch 140 : loss 23.440163 ; accuracy 0.76953334
epoch 160 : loss 22.262396 ; accuracy 0.77845
epoch 180 : loss 21.247375 ; accuracy 0.7863167
epoch 200 : loss 20.369156 ; accuracy 0.79408336
epoch 220 : loss 19.598576 ; accuracy 0.80005
epoch 240 : loss 18.913658 ; accuracy 0.80546665
epoch 260 : loss 18.299295 ; accuracy 0.81098336
epoch 280 : loss 17.744617 ; accuracy 0.81595
epoch 300 : loss 17.239779 ; accuracy 0.81991667
epoch 320 : loss 16.774817 ; accuracy 0.8235833
epoch 340 : loss 16.345797 ; accuracy 0.8268333
epoch 360 : loss 15.947821 ; accuracy 0.8296833
epoch 380 : loss 15.575538 ; accuracy 0.83206666
epoch 400 : loss 15.224714 ; accuracy 0.83468336